%%html
<style>
li{
  margin: 10px 0;
}
</style>
<h2>EDA Analysis Pipeline</h2>
Authors: Stephan, Yijoon, Hans, Frank

<h3>Index</h3>
<ol type="I">
<li><a href='#section1'>Defining the goal/problem</li>  
<li><a href='#section2'>Fetching and cleaning the data</li>  
<li><a href='#section3'>Understand and visualize the data</li>
<li><a href='#section4'>Analyze the data</li>
<li><a href='#section5'>Interpret results</li>
<li><a href='#section6'>Iterate and refine</li>
</ol>

---
<a id='section1'></a>
## 1. Defining the goal/problem
```What is the purpose of this analysis?  ```

Building a model that accurately can predict the lifespan based on features in the data and use that to determine the premium for life insurance in an ethical way.

---
<a id='section2'></a>
## 2. Fetching and cleaning the data
```
- a. collect the data 
- b. check for: errors, missing values(Nan), data types(object,float,int), duplicates and other inconsistencies
- c. clean the data: remove duplicates and remove irrelevant information/columns
```

---
<a id='section3'></a>
## 3. Understand and visualize the data
```
- a. Examine structure and content: size,shape and type of variables
- b. Visualize the data with plots: histograms, box plots, scatter plots and heatmaps
- c. Identify any outliers, patterns, relationships or trends
- d. Decide to impute or delete the outliers
```

---
<a id='section4'></a>
## 4. Analyze the data
```
- a. Apply statistical analysis tools: mean, median, mode, standard deviation
- b. Go through Checklist for linear regression: normal distribution, continuous variable, correlation and p-value
```

---
<a id='section5'></a>
## 5. Interpret the results
```
- Draw conclusions, make insights and communicate in a clear, concise and unbiased manner
```

---
<a id='section5'></a>
## 5. Iterate and refine
```
- Explore alternative approaches e.g. test assumptions and update your conclusions based on feedback and new insights 
```